In [1]:
import pandas as pd
import transformers #4.9

In [2]:
transformers.__version__

'4.2.0'

In [3]:
#!pip install transformers=="4.2"

In [4]:
pd.read_json('ae/laptop/test.json').transpose()

,label,sentence
0,"[B, I, O, O, O, O, O, O, O, O, O, O, O, O, O]","[Boot, time, is, super, fast, ,, around, anywh..."
1,"[B, I, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[tech, support, would, not, fix, the, problem,..."
2,"[O, O, O, O, O, O, O]","[but, in, resume, this, computer, rocks, !]"
3,"[B, I, O, O, O]","[Set, up, was, easy, .]"
4,"[O, O, O, O, O, B, I, O, B, I, O]","[Did, not, enjoy, the, new, Windows, 8, and, t..."
...,...,...
795,"[O, B, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[This, hardware, seems, to, be, better, than, ..."
796,"[O, O, O, O, O, O, O]","[I, 'm, done, with, WinDoze, computers, .]"
797,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B, ...","[I, 've, had, it, for, about, 2, months, now, ..."
798,"[O, O, O, O, O, O, O, B, I, O]","[the, latest, version, does, not, have, a, dis..."


In [5]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

label_list = ['O','B','I']
label_encoding_dict = {'I': 2, 'O': 0, 'B': 1,}

task = "ner" 
model_checkpoint = "bert-base-uncased"
batch_size = 16
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def get_all_tokens_and_ner_tags(directory):
    df = pd.read_json(directory).transpose().reset_index().drop('index', axis=1)
    df = df.rename(columns={"sentence": "tokens", "label": "ner_tags"})
    return df
    
#     return pd.concat([get_tokens_and_ner_tags(os.path.join(directory, filename)) for filename in 
#                       os.listdir(directory)]).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split('\t')[0] for x in y] for y in split_list]
        entities = [[x.split('\t')[1][:-1] for x in y] for y in split_list] 
    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})
  
def get_un_token_dataset(train_directory, test_directory):
    train_df = get_all_tokens_and_ner_tags(train_directory)
    test_df = get_all_tokens_and_ner_tags(test_directory)
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    return (train_dataset, test_dataset)

train_dataset, test_dataset = get_un_token_dataset('ae/rest16/train.json', 
                                                   'ae/rest16/test.json')


In [6]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

Parameter 'function'=<function tokenize_and_align_labels at 0x7f472f3a8670> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### train

In [45]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], 
            "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('rest16_bert.model')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,No log,0.113234,0.668432,0.799747,0.728217,0.954063,1.182000,571.912000
2,No log,0.108398,0.739437,0.798479,0.767824,0.956580,1.186300,569.823000


wandb: WARNING Step must only increase in log calls.  Step 116 < 548; dropping {'eval/loss': 0.11323434859514236, 'eval/precision': 0.6684322033898306, 'eval/recall': 0.7997465145754119, 'eval/f1': 0.7282169648009233, 'eval/accuracy': 0.9540632865875585, 'eval/runtime': 1.182, 'eval/samples_per_second': 571.912, 'train/epoch': 1.0}.
wandb: WARNING Step must only increase in log calls.  Step 232 < 548; dropping {'eval/loss': 0.10839812457561493, 'eval/precision': 0.7394366197183099, 'eval/recall': 0.7984790874524715, 'eval/f1': 0.7678244972577697, 'eval/accuracy': 0.9565803667745415, 'eval/runtime': 1.1863, 'eval/samples_per_second': 569.823, 'train/epoch': 2.0}.
wandb: WARNING Step must only increase in log calls.  Step 232 < 548; dropping {'train/train_runtime': 23.6977, 'train/train_samples_per_second': 9.79, 'train/total_flos': 98848705439160, 'train/epoch': 2.0}.


wandb: WARNING Step must only increase in log calls.  Step 232 < 549; dropping {'eval/loss': 0.10839812457561493, 'eval/precision': 0.7394366197183099, 'eval/recall': 0.7984790874524715, 'eval/f1': 0.7678244972577697, 'eval/accuracy': 0.9565803667745415, 'eval/runtime': 1.1953, 'eval/samples_per_second': 565.544, 'train/epoch': 2.0}.


### predict

In [7]:
tokenizer = AutoTokenizer.from_pretrained('./rest16_bert.model/')

In [8]:
model = AutoModelForTokenClassification.from_pretrained('./rest16_bert.model/', num_labels=len(label_list))

In [18]:
results = []
labels = []
model.eval()
with torch.no_grad():
    for i in test_tokenized_datasets:
        res = list(np.argmax(model.forward(torch.tensor(i['input_ids']).unsqueeze(0).to('cpu'), 
                      torch.tensor(i['attention_mask']).unsqueeze(0).to('cpu'))[0][0].numpy()[1:-1],1))
        resint = [int(arg) for arg in res]
        results.append(resint)
        labels.append(list(np.array(i['labels'][1:-1])))

In [10]:
import json

In [50]:
json_string = json.dumps({"rest16_bert_base_preds":results})
with open('rest16_bert_base_preds.json', 'w') as outfile:
    outfile.write(json_string)

### f1_score

In [11]:
results = []
labels = []
model.eval()
with torch.no_grad():
    for i in test_tokenized_datasets:
        results+=list(np.argmax(model.forward(torch.tensor(i['input_ids']).unsqueeze(0).to('cpu'), 
                      torch.tensor(i['attention_mask']).unsqueeze(0).to('cpu'))[0][0].numpy()[1:-1],1))
        labels+=list(np.array(i['labels'][1:-1]))

In [14]:
from sklearn.metrics import f1_score, confusion_matrix

In [16]:
f1_score(labels,results,labels =[1,2],average='macro')

0.7696753195205208

In [15]:
confusion_matrix(labels,results)

array([[9702,  155,  101],
       [ 116,  655,   18],
       [  77,   16,  284]])

# evaluation

In [28]:
from sklearn.metrics import precision_recall_fscore_support
import json

def get_chunk_type(tok, idx_to_tag):

    tag_name = idx_to_tag[tok]
    tag_class = tag_name.split('-')[0]
    tag_type = tag_name.split('-')[-1]
    return tag_class, tag_type


def get_chunks(seq, tags,message=None):

    default = tags['O']
    idx_to_tag = {idx: tag for tag, idx in tags.items()}
    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        # End of a chunk 1
        if tok == default and chunk_type is not None:
            # Add a chunk.
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None

        # End of a chunk + start of a chunk!
        elif tok != default:
            tok_chunk_class, tok_chunk_type = get_chunk_type(tok, idx_to_tag)
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk_type, i
            elif tok_chunk_type != chunk_type or tok_chunk_class == 'B':
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk_type, i
        else:
            pass

    # end condition
    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)
    #print(message + "{}{}{}".format(seq, tags, chunks))

    return chunks

def run_evaluate(labels, labels_pred, sequence_lengths):

    accs = []
    correct_preds, total_correct, total_preds = 0., 0., 0.
    gold = []
    pred = []
    acc_list = []

    for lab, lab_pred, length in zip(labels, labels_pred,
                                        sequence_lengths):
        lab      = lab[:length]
        lab_pred = lab_pred[:length]
        acc_list.append(lab==lab_pred)
        gold+=lab
        pred+=lab_pred
        accs    += [a==b for (a, b) in zip(lab, lab_pred)]
#         lab_chunks      = set(get_chunks(lab, config.vocab_tags,message = "gold standard"))
#         #tmp_lab_chunks.append(lab_chunks)
#         lab_pred_chunks = set(get_chunks(lab_pred,config.vocab_tags,message = "prediction"))
#         #tmp_lab_pred_chunks.append(lab_pred_chunks)
#         correct_preds += len(lab_chunks & lab_pred_chunks)
#         total_preds   += len(lab_pred_chunks)
#         total_correct += len(lab_chunks)

#     p   = correct_preds / total_preds if correct_preds > 0 else 0
#     r   = correct_preds / total_correct if correct_preds > 0 else 0
#     f1  = 2 * p * r / (p + r) if correct_preds > 0 else 0

    acc = np.mean(accs)

    #include 1,2
    score = precision_recall_fscore_support(gold, pred, labels =[1,2], average='macro')
    #include 0,1,2
    score_0 = precision_recall_fscore_support(gold, pred, average='macro')

    return acc, ["{0:.5f}".format(x) for x in score[:3]], ["{0:.5f}".format(x) for x in score_0[:3]], acc_list

In [26]:
sequence_lengths = [len(x) for x in labels]

In [27]:
run_evaluate(labels, results, sequence_lengths)

(0.9565803667745415,
 ['0.74885', '0.79174', '0.76968'],
 ['0.82606', '0.85259', '0.83891'],
 [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  True,
  True,
  True,
  T